![My Image](https://raw.githubusercontent.com/ralf-42/Image/main/genai-banner-2.jpg)

<p><font size="5" color='grey'> <b>
Grundlagen Modellansteuerung
</b></font> </br></p>

---

In [ ]:
#@title
#@markdown   <p><font size="4" color='green'>Umgebung einrichten</font> </br></p>
!uv pip install --system --prerelease allow -q git+https://github.com/ralf-42/genai_lib
from genai_lib.utilities import check_environment, get_ipinfo, setup_api_keys, mprint
setup_api_keys(['OPENAI_API_KEY', 'HF_TOKEN'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

# 1 | Überblick
---


Die rasante Entwicklung im Bereich der künstlichen Intelligenz (KI) hat zu beeindruckenden Fortschritten bei großen Sprachmodellen (LLMs) geführt. Diese Modelle können menschenähnlichen Text generieren, Sprachen übersetzen und komplexe Fragen beantworten. Doch wie genau steuert man diese Modelle, um die gewünschten Ergebnisse zu erzielen? In diesem Modul werden drei gängige Methoden der Modellansteuerung untersucht: Prompting, Retrieval-Augmented Generation (RAG) und Fine-Tuning.



<img src="https://raw.githubusercontent.com/ralf-42/Image/main/rag_klein.jpg" width="500
" alt="Avatar">

🔄 **1. Prompting**

* **Was passiert?**    
  Das Modell wird **nicht verändert**, sondern durch **clever gestaltete Eingaben** (Prompts) zu einer gewünschten Leistung gebracht.

* **Modellanpassung?**   
  ✅ *indirekte, temporäre Anpassung*   
  ❌ *keine Änderung des Modells selbst*

* **Beispiel:**
  *„Du bist ein Experte für Medizinrecht. Antworte nur mit rechtlich abgesicherten Aussagen.“*

* **Vorteil:** Schnell, flexibel, keine Trainingskosten

* **Nachteil:** Begrenzte Steuerbarkeit, kein echtes „Lernen“





📚 **2. RAG (Retrieval-Augmented Generation)**

* **Was passiert?**   
  Das Modell bekommt Zugriff auf externe Wissensquellen (z. B. PDF, Datenbank, Vektorensuche) – diese Infos werden **zur Laufzeit in den Prompt eingefügt**.

* **Modellanpassung?**   
  ✅ *strukturelle Anpassung durch Architektur*   
  ❌ *Modell selbst bleibt unverändert*

* **Beispiel:**   
  Eine Frage zu internen Firmenrichtlinien wird durch Retrieval aus dem Firmen-Wiki beantwortet.

* **Vorteil:** Aktuelle & domänenspezifische Infos, ohne das Modell zu verändern

* **Nachteil:** Qualität hängt stark von der Dokumentenstruktur und Vektorsuche ab





🧠 **3. Fine-Tuning (Nachtraining auf Basisdaten)**

* **Was passiert?**   
  Das Modell wird mit eigenen Beispielen **nachtrainiert**, um z. B. den Tonfall, Fachsprache oder Verhalten gezielt zu ändern.

* **Modellanpassung?**   
  ✅ *tiefe, permanente Modellanpassung*   
  🛠️ *reelles „Lernen“ auf Parameter-Ebene*

* **Beispiel:**   
  Ein Modell wird mit tausenden juristischen Fällen nachtrainiert, um bessere Schriftsätze zu erzeugen.

* **Vorteil:** Sehr genaue Steuerung möglich, konsistentes Verhalten

* **Nachteil:** Aufwändig, teuer, benötigt viele Daten + GPU-Ressourcen




| Technik         | Modellparameter<br> geändert? | Anpassungstiefe        | Beispiel                           |
| --------------- | ------------------------- | ---------------------- | ---------------------------------- |
| **Prompting**   | ❌ nein                    | leicht (oberflächlich) | Rollenanweisung im Prompt          |
| **RAG**         | ❌ nein                    | mittel (strukturell)   | Infos aus PDF automatisch einfügen |
| **Fine-Tuning** | ✅ ja                      | tief (parametrisch)    | Modell kennt neue Fachbegriffe     |


# 2 | Prompting
---

Prompting ist die einfachste und direkteste Methode zur Steuerung von LLMs. Dabei wird dem Modell eine Textaufforderung, der sogenannte `Prompt`, gegeben, der die gewünschte Ausgabe beschreibt. Die Qualität der Ausgabe hängt stark von der Qualität des Prompts ab. Ein gut formulierter Prompt sollte klar, prägnant und spezifisch sein.


[Say What You See](https://artsandculture.google.com/experiment/say-what-you-see/jwG3m7wQShZngw)

<p><font color='black' size="5">
Prompt-Elemente
</font></p>

Eingabeaufforderungen bestehen in der Regel aus vier wesentlichen Elementen, die je nach Aufgabe variieren können. Die Hauptbestandteile sind:

* **Anweisungen** – Definieren die Aufgabe des Modells, entweder als Aufgabenbeschreibung oder als konkrete Handlungsanweisung.  
* **Kontext** – Liefert ergänzende Informationen, um das Modell bei der Bearbeitung zu unterstützen.  
* **Eingabedaten** – Enthalten die spezifischen Informationen, zu denen eine Antwort generiert werden soll.  
* **Ausgabeindikator** – Signalisiert den Übergang zur erwarteten Modellantwort.  

Weitere Beispiele zur Anwendung dieser Struktur folgen später.

Nachfolgend ein Beispiel für eine Eingabeaufforderung zur Erstellung einer Produktzusammenfassung.

```
Fassen Sie die folgende Produktbewertung in einem Satz zusammen:
Produkt: Intelligente digitale Personenwaage für Körpergewicht, Fett, BMI, Muskelmassezusammensetzung
Produktbewertung: Diese Waage ist fantastisch! Der Einrichtungsvorgang war schnell und einfach und
dauerte nur wenige Minuten. Ich konnte die App kostenlos auf mein etwas veraltetes iPhone 8
herunterladen. Die Funktion, bei der Ihr Gewicht sofort mit der App synchronisiert wird, macht die
Nachverfolgung mühelos. Die Funktionen zur Essensplanung und Kalorienzählung der App sind unglaublich
benutzerfreundlich. Ich bin absolut begeistert! Außerdem hat die Waage ein elegantes, modernes
Erscheinungsbild, das wirklich attraktiv ist.


Zusammenfassung: ...
```

Diese Eingabeaufforderung erzeugt möglicherweise die folgende Ausgabe:

```
Die intelligente digitale Personenwaage wird für ihre schnelle Einrichtung, die einfache
Synchronisierung mit ihrer App auf älteren Telefonen zur mühelosen Nachverfolgung, die
benutzerfreundlichen Funktionen zur Essensplanung und Kalorienzählung sowie ihr schlankes, modernes
Design hoch gelobt.
```

<p><font color='black' size="5">
Code-Beispiel
</font></p>

In [ ]:
from openai import OpenAI, chat

client = OpenAI()

user_input = """
Fassen Sie die folgende Produktbewertung in einem Satz zusammen:
Produkt: Intelligente digitale Personenwaage für Körpergewicht, Fett, BMI, Muskelmassezusammensetzung
Produktbewertung: Diese Waage ist fantastisch! Der Einrichtungsvorgang war schnell und einfach und
dauerte nur wenige Minuten. Ich konnte die App kostenlos auf mein etwas veraltetes iPhone 8
herunterladen. Die Funktion, bei der Ihr Gewicht sofort mit der App synchronisiert wird, macht die
Nachverfolgung mühelos. Die Funktionen zur Essensplanung und Kalorienzählung der App sind unglaublich
benutzerfreundlich. Ich bin absolut begeistert! Außerdem hat die Waage ein elegantes, modernes
Erscheinungsbild, das wirklich attraktiv ist.
"""

completion = chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "Du bist ein hilfreicher KI-Assistent."},
    {"role": "user", "content": user_input}
  ]
)

mprint('## 🤖 KI:')
mprint(completion.choices[0].message.content)

# 3 | Retrieval-Augmented Generation
---

RAG (Retrieval-Augmented Generation) ist eine **hybride Methode**, die die Stärken von **LLMs** mit denen von **Information-Retrieval-Systemen** **kombiniert**. Dabei greift das LLM auf llm-externe Wissensdatenbanken zu, um seine Antworten zu verbessern und die Probleme von Halluzinationen und veralteten Informationen zu mindern.

**Funktionsweise:**

+ Der Benutzer gibt eine Anfrage in das RAG-System ein.
+ Das System sucht in externe Wissens nach relevanten Informationen.
+ Die relevanten Informationen werden dem LLM als Kontext bereitgestellt.
+ Das LLM generiert eine Antwort basierend auf der Anfrage **und** dem Kontext.


**Einsatzszenarien:**

RAG eignet sich besonders für Aufgaben, die aktuelle oder domänenspezifische Informationen erfordern, wie z. B.:

+ Kundensupport mit Zugriff auf Produktdatenbanken
+ Medizinische Diagnose mit Zugriff auf aktuelle Forschungsergebnisse
+ Finanzplanung mit Zugriff auf Marktdaten
+ Beantwortung von Fragen zu Unternehmensinformationen



# 4 | Fine-Tuning
---

Fine-Tuning ist eine Methode, bei der ein bereits **trainiertes LLM** auf einer kleineren, **aufgabenspezifischen** Datenmenge weiter trainiert wird. Dadurch kann das Modell an **spezifische** **Anforderungen** angepasst und seine Leistung für diese Aufgaben verbessert werden.

**Funktionsweise:**

+ Ein vortrainiertes LLM wird ausgewählt.
+ Eine aufgabenspezifische Datenmenge wird vorbereitet.
+ Das LLM wird auf dieser Datenmenge trainiert.
+ Die Parameter des LLM werden angepasst, um die Leistung für die spezifische Aufgabe zu optimieren.


**Einsatzszenarien:**

Fine-Tuning eignet sich besonders für Aufgaben, die eine hohe Genauigkeit und **Domänenspezialisierung** erfordern, wie z. B.:

+ fachlich spezialisierte LLMs (Recht, Medizin, ...)
+ Sentimentanalyse
+ Textklassifizierung
+ Spam-Erkennung
+ Personalisierte Kundeninteraktionen




# 5 | Entscheidungskriterien
---

Die Wahl der geeigneten Methode hängt von verschiedenen Faktoren ab, darunter:


| Kriterium | Prompting | RAG | Fine-Tuning |
|-----------|-----------|-----|-------------|
| **Komplexität** | Niedrig<br><br>Die Implementierung ist <br>einfach und erfordert keine <br>Änderungen am Modell. | Mittel<br><br>RAG erfordert die Integration<br>von Retrieval-Mechanismen<br>und einer Wissensdatenbank. | Hoch<br><br>Fine-Tuning beinhaltet<br>einen komplexeren Prozess<br>des weiteren Trainings. |
| **Effizienz** | Hoch<br><br>Prompting ermöglicht eine<br>schnelle und flexible<br>Interaktion mit dem Modell. | Mittel<br><br>RAG benötigt zusätzliche<br>Schritte für den Datenabruf,<br>was die Effizienz mindert. | Niedrig<br><br>Fine-Tuning ist rechen-<br>intensiv und zeitaufwendig. |
| **Genauigkeit** | Niedrig<br><br>Die Genauigkeit hängt stark<br>von der Qualität des<br>Prompts ab. | Mittel<br><br>RAG bietet höhere<br>Genauigkeit durch externe<br>Informationen. | Hoch<br><br>Führt zu höherer<br>Genauigkeit bei<br>spezifischen Aufgaben. |
| **Flexibilität** | Hoch<br><br>Sehr flexibel und für<br>verschiedene Aufgaben<br>einsetzbar. | Mittel<br><br>Flexibler als Fine-Tuning,<br>aber weniger flexibel als<br>Prompting. | Niedrig<br><br>Ist auf bestimmte Aufgaben<br>spezialisiert und weniger<br>anpassungsfähig. |
| **Ressourcen-<br>bedarf** | Niedrig<br><br>Benötigt nur minimale<br>Ressourcen für die<br>Ausführung. | Mittel<br><br>Ressourcen für Datenbank-<br>verwaltung und Retrieval<br>erforderlich. | Hoch<br><br>Erfordert erhebliche<br>Rechenleistung und Zeit<br>für Training. |
| **Datenbedarf** | Niedrig<br><br>Keine zusätzlichen<br>Trainingsdaten<br>erforderlich. | Mittel<br><br>Benötigt Wissensdatenbank<br>mit relevanten<br>Informationen. | Hoch<br><br>Benötigt große Mengen an<br>spezifischen<br>Trainingsdaten. |
| **Beschreibung** | Einfache Methode mit<br>Textanweisungen für das<br>Modell. | Kombination von LLMs mit<br>Information-Retrieval-<br>Systemen. | Weiteres Training des<br>Modells auf spezifischen<br>Datensätzen. |

